<a href="https://colab.research.google.com/github/tajtu1406/BT2-N15/blob/master/Speech.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [26]:
import librosa, librosa.display
import matplotlib.pyplot as plt
import numpy as np
import IPython.display as idsp

# MFCC Feature calculation

In [27]:
import os
import numpy as np
import librosa


def mfcc(audio_path):
    y, sr = librosa.load(audio_path)
    # MFCCs
    mfccs = librosa.feature.mfcc(y = y, sr = sr, n_mfcc = 13)
    ans = [mfccs]
    # Compute the first MFCCs derivatives
    delta_mfccs = librosa.feature.delta(mfccs, order = 1, mode ='nearest')
    ans.append(delta_mfccs)
    # Compute the second MFCCs derivatives
    delta2_mfccs = librosa.feature.delta(mfccs, order = 2, mode ='nearest')
    ans.append(delta2_mfccs) 
    return np.transpose(np.concatenate(ans, axis = 0),[1,0])

# DTW Implementation 
### Averaging trains template

In [4]:
from math import sqrt

def calculate_Euclid_distance(A, B):
    return sqrt(sum([(a - b) ** 2 for (a, b) in zip(A, B)]))


def dtw(M1, M2):
    # length of two sequences
    M1_len = len(M1)
    M2_len = len(M2)
    cost_0 = np.zeros((M1_len + 1, M2_len + 1))
    cost_0[0, 1:] = np.inf
    cost_0[1:, 0] = np.inf
    # Initialize the array size to M1_len * M2_len
    cost = cost_0[1:, 1:]
    for i in range(M1_len):
        for j in range(M2_len):
            cost[i, j] = calculate_Euclid_distance(M1[i], M2[j])
    # DP to calculate cost matrix
    for i in range(M1_len):
        for j in range(M2_len):
            cost[i, j] += min([cost_0[i, j], \
                               cost_0[min(i + 1, M1_len - 1), j], \
                               cost_0[i, min(j + 1, M2_len - 1)]])

    # calculate the warp path
    if len(M1) == 1:
        path = np.zeros(len(M2)), range(len(M2))
    elif len(M2) == 1:
        path = range(len(M1)), np.zeros(len(M1))
    else:
        i, j = np.array(cost_0.shape) - 2
        path_1, path_2 = [i], [j]
        while (i > 0) or (j > 0):
            arg_min = np.argmin((cost_0[i, j], cost_0[i, j + 1], cost_0[i + 1, j]))
            if arg_min == 0:
                i -= 1
                j -= 1
            elif arg_min == 1:
                i -= 1
            else:
                j -= 1
            path_1.insert(0, i)
            path_2.insert(0, j)
        path = np.array(path_1), np.array(path_2)
    # the minimum distance is the normalized distance
    return cost[-1, -1] / sum(cost.shape), path


def train_model_dtw(train_dir):
    model = []
    for label in ["len", "xuong", "trai", "phai", "a", "b", "nhay", "ban"]:
        label_dir = os.path.join(train_dir, str(label))
        file_list = os.listdir(label_dir)
        mfcc_list = []

        # read input files and calculate MFCC features
        for j in range(len(file_list)):
            file_path = os.path.join(label_dir, file_list[j])
            if file_path[-4:] == ".wav":
              file_mfcc = mfcc(file_path)
              mfcc_list.append(file_mfcc)

        # set the first sequence as master
        mfcc_count = np.zeros(len(mfcc_list[0]))
        mfcc_all = np.zeros(mfcc_list[0].shape)
        for i in range(len(mfcc_list)):
            # calculate the wrap path between master and each template
            _, path = dtw(mfcc_list[0], mfcc_list[i])
            for j in range(len(path[0])):
                mfcc_count[int(path[0][j])] += 1
                mfcc_all[int(path[0][j])] += mfcc_list[i][path[1][j]]

        # Generalization by averaging the templates
        final_mfcc = np.zeros(mfcc_all.shape)
        for i in range(len(mfcc_count)):
            for j in range(len(mfcc_all[i])):
                final_mfcc[i][j] = mfcc_all[i][j] / mfcc_count[i]
        model.append(final_mfcc)
    return model


def predict_dtw(model, file_path):
    mfcc_feat = mfcc(file_path)
    result = 0
    min_dist, _ = dtw(model[0], mfcc_feat)
    label = ["len", "xuong", "trai", "phai", "a", "b", "nhay", "ban"]
    for i in range(len(model)):
        dist, _ = dtw(model[i], mfcc_feat)
        if dist < min_dist:
            result = i
            min_dist = dist
    return label[result]

In [28]:
train_dir = "/content/drive/MyDrive/15 (nhóm có stt 14 trong danh sách)/trainDTW"

model = train_model_dtw(train_dir)

In [29]:
pred_dir = "/content/drive/MyDrive/15 (nhóm có stt 14 trong danh sách)/test/file128.wav"
sound, sr = librosa.load(pred_dir)

pred = predict_dtw(model, pred_dir)
print(pred)
idsp.Audio(data=sound, rate=sr)

nhay


# HMM

In [ ]:
!pip install hmmlearn

In [11]:
from hmmlearn import hmm
import numpy as np

def train_model_hmm(train_dir):
    hmm_models = []
    # iterate through train dir
    for label in os.listdir(train_dir):
        label_dir = os.path.join(train_dir, label)
        
        # start training
        X = np.array([])
        train_files = [x for x in os.listdir(label_dir) if x.endswith('.wav')]
        for file_name in train_files:
            file_path = os.path.join(label_dir, file_name)
            print(label, file_path)
            try:
              features_mfcc = mfcc(file_path)
            
              # append mfcc to X
              if len(X) == 0:
                  X = features_mfcc
              else:
                  X = np.append(X, features_mfcc, axis=0)
            except:
              continue
              
        # HMM with Mixture of Gaussians
        model = hmm.GMMHMM(n_components=4, n_iter=1000)

        # fit model
        np.seterr(all='ignore')
        model.fit(X)
        hmm_models.append((model, label))
    return hmm_models


def predict_hmm(hmm_models, test_file):
    # get mfcc features
    features_mfcc = mfcc(test_file)

    # calculate the score and get the maximum score
    max_score = -float('inf')
    result = ""
    for item in hmm_models:
        model, label = item
        score = model.score(features_mfcc)
        if score > max_score:
            max_score = score
            result = label
    return result
    

In [ ]:
train_path = "/content/drive/MyDrive/15 (nhóm có stt 14 trong danh sách)/train"
hmm_models = train_model_hmm(train_path)

In [30]:
pred_dir = "/content/drive/MyDrive/15 (nhóm có stt 14 trong danh sách)/test/file128.wav"
sound, sr = librosa.load(pred_dir)
pred = predict_hmm(hmm_models, pred_dir)
print(pred)
idsp.Audio(data=sound, rate=sr)

nhay
